In [1]:
import json
from urllib.request import urlopen
import pandas as pd
from pulp import *
import re
from collections import Counter

In [2]:
#Set number of available players
formations = [
    {'GKP': 1, 'DEF': 3, 'MID': 4, 'FWD': 3},
    {'GKP': 1, 'DEF': 3, 'MID': 5, 'FWD': 2},
    {'GKP': 1, 'DEF': 4, 'MID': 5, 'FWD': 1},
    {'GKP': 1, 'DEF': 4, 'MID': 4, 'FWD': 2},
    {'GKP': 1, 'DEF': 4, 'MID': 3, 'FWD': 3} 
]

sub_formations= [
    {'GKP': 1, 'DEF': 1, 'MID': 1, 'FWD': 1},
    {'GKP': 1, 'DEF': 2, 'MID': 0, 'FWD': 1}
]

#Set Budget

BUDGET = 90
SUB_BUDGET = 10
PLAYER_TEAM_LIMIT = 3

In [3]:
class Dataset():
    
    def get_url(self, key, Id=None):
        base_url = 'https://fantasy.premierleague.com/drf'
        api = {
            'All': '/bootstrap-static',
            'Player': '/element-summary/{}',
            'Team':'/my-team/48049/{}',
            '1':'/bootstrap',
            '2':'/bootstrap-static',
            '3':'/bootstrap-dynamic',
            '4':'/events',
            'player_stats':'/elements',
            'player_positions':'/element-types',
            '7':'/fixtures',
            'teams':'/teams',
            '9':'/region',
            '10':'/transfers',
            '11':'/entry/{}',
            '12':'/entries',
            '13':'/my-team/{}',
            '14':'/leagues-entered/{}',
            '15':'/leagues-classic/{}',
            '16':'/leagues-classic-standings/{}',
            '17':'/leagues-h2h/{}',
            '18':'/leagues-h2h-standings/{}'
        }

        if Id is None:
            url = ''.join([base_url, api[key]])
        else:
            api_extension = api[key]
            api_extension = api_extension.format(Id)
            url = ''.join([base_url, api_extension])
        return url

    def pull_data(self, key, Id=None):
        json_data = json.load(urlopen(self.get_url(key, Id)))
        df = pd.read_json(json.dumps(json_data))
        return df

    def get_player_data(self, active=False):
        player_stats = self.pull_data('player_stats')
        player_positions = self.pull_data('player_positions')[['id', 'singular_name_short']]
        player_teams = self.pull_data('teams')[['code', 'name']]
        
        player_table = player_stats.merge(player_positions, left_on='element_type', right_on='id')
        player_table = player_table.merge(player_teams, left_on='team_code', right_on='code')
        player_table['now_cost'] = player_table['now_cost']/10
        if active:
            player_table = player_table.loc[player_table['chance_of_playing_next_round'] == 100]
            player_table = player_table.reset_index()
        return player_table
    
    def model_data(self):
        active_players = self.get_player_data(True)
        player_pool = active_players[['web_name', 'singular_name_short', 'now_cost', 'total_points', 'name']]
        player_pool.columns = ['web_name', 'position', 'cost', 'total_points', 'team_name']
        
        costs = {}
        points = {}

        
        for team in player_pool.team_name.unique():
            team_players = player_pool.loc[player_pool.team_name == team]
            t_costs = {}
            t_points = {}

            for pos in player_pool.position.unique():
                pos_players = team_players.loc[team_players.position == pos]
                t_costs[pos] = list(pos_players[['web_name', 'cost']].set_index('web_name').to_dict().values())[0]
                t_points[pos] = list(pos_players[['web_name', 'total_points']].set_index('web_name').to_dict().values())[0]
            costs[team] = t_costs
            points[team] = t_points

        return (costs, points)
        

In [4]:
class TeamSolutions():
    def __init__(self, data, formations, budgets):
        self.formations = formations
        self.budgets = budgets
        self.costs, self.points = data
#         self._vars = {pos: LpVariable.dict(pos, players, cat="Binary") for pos, players in self.points.items()}
        self._vars = {team: {pos: LpVariable.dict(pos, players, cat="Binary") for pos, players in positions.items()} for team, positions in self.points.items()}
        
    def solve_problem(self, formation, budget):
        #Starting the problem

        prob = LpProblem('Naive_total_score_max', LpMaximize)
        rewards = []
        Costs = []
        position_constraints = []

        #Reward
        for team, position in self._vars.items():
            Team = []
            for pos, players in position.items():
                Costs += lpSum([self.costs[team][pos][player] * self._vars[team][pos][player] for player in players])
                rewards += lpSum([self.points[team][pos][player] * self._vars[team][pos][player] for player in players])
                prob += lpSum([self._vars[all_team][pos][player] for player in players])
                Team += lpSum([self._vars[team][pos][player] for player in players]) 
            prob += lpSum(Team) <= 3
            
        prob += lpSum(rewards)
        prob += lpSum(Costs) <= budget
        
        prob.solve()
        return prob
   
    def print_summary(self, prob):
        div = '----------------------------\n'
        print("Variables:\n")
        score = str(prob.objective)
        constraints = [str(const) for const in prob.constraints.values()]
        for v in prob.variables():
            score = score.replace(v.name, str(v.varValue))
            constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
            if v.varValue != 0:
                print(v.name, "=", v.varValue)
        print(div)
        print("Constraints:")
        for constraint in constraints:
            constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
            if constraint_pretty != "":
                print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
        print(div)
        print("Score:")
        score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
        print("{} = {}".format(score_pretty, eval(score)))
    
    def summarise(self, prob):
        Players = []
        Budget = {}
        Score = {}
        
        score = str(prob.objective)
        constraints = [str(const) for const in prob.constraints.values()]
        
        for v in prob.variables():
            score = score.replace(v.name, str(v.varValue))
            constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
            if v.varValue != 0:
                Players.append(v.name)


#         for player in Players:
#             position = player[0:3]
#             try:
#                 web_name = player[4:].replace("_"," ")
#                 Budget[player] = self.costs[position][web_name]
#                 Score[player] = self.points[position][web_name]
               
#             except:
#                 web_name = player[4:].replace("_","-")
#                 Budget[player] = self.costs[position][web_name]
#                 Score[player] = self.points[position][web_name]
                

#         Budget['Total'] = sum(Budget.values())
#         Score['Total'] = sum(Score.values())
#         Teams['Totals'] = Counter(Teams.values())
        return {'Players': Players}#,'Budget': Budget,'Score': Score}

    def solve_for_all_formations(self):
        possible_teams = []
        for formation in self.formations:
            possible_team = self.solve_problem(formation, self.budgets)
            team_summary = self.summarise(possible_team)
            possible_teams.append(team_summary)
        return possible_teams
            
            

In [5]:
dataset = Dataset()
data = dataset.model_data()
costs, points = data

In [6]:
costs

{'Arsenal': {'GKP': {'Cech': 4.9},
  'DEF': {'Bellerín': 5.4, 'Kolasinac': 4.9, 'Sokratis': 5.1},
  'MID': {'Özil': 8.3,
   'Ramsey': 7.3,
   'Iwobi': 5.5,
   'Elneny': 4.3,
   'Mkhitaryan': 6.7,
   'Maitland-Niles': 4.4,
   'Torreira': 4.9,
   'Guendouzi': 4.5},
  'FWD': {'Aubameyang': 10.9}},
 'Bournemouth': {'GKP': {'Begovic': 4.5},
  'DEF': {'Francis': 4.4, 'Daniels': 4.2, 'Aké': 5.0},
  'MID': {'Pugh': 4.3, 'Stanislas': 6.0, 'Fraser': 6.2},
  'FWD': {'Mousset': 4.9, 'King': 6.3}},
 'Brighton': {'GKP': {'Button': 4.0},
  'DEF': {'Dunk': 4.5, 'Duffy': 4.7, 'Bruno': 4.4, 'Bong': 4.5},
  'MID': {'Groß': 6.7, 'Pröpper': 4.8, 'Izquierdo': 5.9},
  'FWD': {'Locadia': 5.3, 'Andone': 5.0}},
 'Burnley': {'GKP': {'Heaton': 4.8},
  'DEF': {'Bardsley': 4.2},
  'MID': {'Brady': 5.4, 'Westwood': 4.5, 'Cork': 4.7, 'Lennon': 5.0},
  'FWD': {'Barnes': 5.7, 'Wood': 6.2, 'Vydra': 5.8}},
 'Cardiff': {'GKP': {},
  'DEF': {'Morrison': 4.7, 'Peltier': 3.8},
  'MID': {'Arter': 4.9,
   'Ralls': 4.7,
   'Hoi

In [7]:
team_model = TeamSolutions(data, formations, BUDGET)

In [8]:
possible_teams = team_model.solve_for_all_formations()

In [9]:
possible_teams

[{'Players': ['DEF_David_Luiz',
   'DEF_Doherty',
   'DEF_Duffy',
   'DEF_Dunk',
   'DEF_Gomez',
   'DEF_Keane',
   'DEF_Laporte',
   'DEF_Rüdiger',
   'DEF_Schindler',
   'DEF_Wan_Bissaka',
   'DEF_van_Dijk',
   'FWD_Jiménez',
   'GKP_Begovic',
   'GKP_Dubravka',
   'GKP_McCarthy',
   'MID_Fraser',
   'MID_Sigurdsson']},
 {'Players': ['DEF_David_Luiz',
   'DEF_Doherty',
   'DEF_Duffy',
   'DEF_Dunk',
   'DEF_Gomez',
   'DEF_Keane',
   'DEF_Laporte',
   'DEF_Rüdiger',
   'DEF_Schindler',
   'DEF_Wan_Bissaka',
   'DEF_van_Dijk',
   'FWD_Jiménez',
   'GKP_Begovic',
   'GKP_Dubravka',
   'GKP_McCarthy',
   'MID_Fraser',
   'MID_Sigurdsson']},
 {'Players': ['DEF_David_Luiz',
   'DEF_Doherty',
   'DEF_Duffy',
   'DEF_Dunk',
   'DEF_Gomez',
   'DEF_Keane',
   'DEF_Laporte',
   'DEF_Rüdiger',
   'DEF_Schindler',
   'DEF_Wan_Bissaka',
   'DEF_van_Dijk',
   'FWD_Jiménez',
   'GKP_Begovic',
   'GKP_Dubravka',
   'GKP_McCarthy',
   'MID_Fraser',
   'MID_Sigurdsson']},
 {'Players': ['DEF_David_Lui

In [10]:
sub_model = TeamSolutions(data, sub_formations, SUB_BUDGET)

In [11]:
possible_subs = sub_model.solve_for_all_formations()

In [ ]:
positions = ['gkp', 'fwd']
clubs = ['this', 'si', 'club']
